In [ ]:
import requests
import pandas as pd
# import polyline
import json
import pickle


In [ ]:
!pip install polyline


In [ ]:
url='https://drive.google.com/file/d/1gKZ4bvwwVjyvrIZorG2zBEswDQpcwW88/view?usp=sharing'
file_id=url.split('/')[-2]
url='https://drive.google.com/uc?id=' + file_id
infodf = pd.read_csv(url,index_col=0)

In [ ]:
infodf

In [ ]:
disdict = {}
locationarray =infodf['name'].unique()
for i in range(len(locationarray)):
  disdict[locationarray[i]]={}
  j=i+1
  while j < len(locationarray):
    disdict[locationarray[i]][locationarray[j]]=0
    j+=1

In [ ]:
def finddis(l1,l2):
  
  url=f'https://maps.googleapis.com/maps/api/distancematrix/json?units=imperial&origins={l1[0]},{l1[1]}&destinations={l2}&key=APIKEY'
  res = json.loads(requests.get(url).content)
  # if distance.distance(l1,l2).km > 50 :
  #   print(l1,l2)
  # print(distance.distance(l1,l2).km)
  return res

In [ ]:
for index ,row in infodf.iterrows():
    dis=[]
    name1 = row['name']
    lat1 = row['lat']
    long1 =row['long']
    des=""
    r=0
    for l in range(len(locationarray)):
      lat2 =infodf.loc[l]['lat']
      long2 =infodf.loc[l]['long']
      if  r<24 and l!=len(locationarray)-1:
        en =polyline.encode([(lat2,long2)], 5)
        des+=f"enc:{en}:"+"|"
        
        
      if r ==24 or l==len(locationarray)-1:
        en =polyline.encode([(lat2,long2)], 5)
        des+=f"enc:{en}:"+"|"
        dis.append(finddis((lat1,long1),des))
        des=""
        r=0
      r+=1
      # l+=1


    finaldis =[]
    for i in dis:
      res = i['rows'][0]['elements']
      for j in res:
        # print(j['distance'])
        distance = j['distance']['text']
        duration = j['duration']['text']
        finaldis.append({'distance':distance,'duration':duration})

    # k =row.Index+1
    for k in range(len(locationarray)):
        # print(k)
      # for kk in range(len(finaldis)):
        # if k != row.Index:
        name2=infodf.loc[k]['name']
          # lat2 =locationinfodf.loc[k]['lat']
          # long2 =locationinfodf.loc[k]['long']
          # print(name2,lat2,long2)
          # dis = finddis((lat1,long1),(lat2,long2))
        disdict[name1][name2] = finaldis[k]


In [ ]:
p = pd.DataFrame()
a=[]
for n in disdict:
  for i,v in disdict[n].items():
    a.append({'from':n, 'to':i,'distance':v['distance'],'duration':v['duration']})
p = p.append(a)


In [ ]:
p['duration'].value_counts()

17 mins           4624
18 mins           4600
19 mins           4589
16 mins           4457
20 mins           4149
                  ... 
1 hour 39 mins       1
1 hour 32 mins       1
1 hour 30 mins       1
1 hour 35 mins       1
1 hour 34 mins       1
Name: duration, Length: 98, dtype: int64

In [ ]:
for i,r in p.iterrows():
  if r['from'] == r['to']:
    r['distance'] = 0
    r['duration'] = 0